In [60]:
from simple_lama_inpainting import SimpleLama
from PIL import Image, ImageChops, ImageDraw
import numpy as np
import cv2
import os

In [69]:
# 1. Lama

In [65]:
def narrow_mask(mask, iterations=1):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    narrowed_mask = mask_array
    for _ in range(iterations):
        narrowed_mask = cv2.erode(narrowed_mask, kernel, iterations=1)
    return Image.fromarray(narrowed_mask)

# Funktion, um die Maske zu erweitern (Dilation)
def expand_mask(mask, iterations=1):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    expanded_mask = mask_array
    for _ in range(iterations):
        expanded_mask = cv2.dilate(expanded_mask, kernel, iterations=1)
    return Image.fromarray(expanded_mask)

In [66]:

# Liste von Bild- und Masken-Pfaden
image_paths = [
    "images/ADE_00432_aquarium.jpg",
    "images/ADE_train_00022632.jpg",
    "images/ADE_train_00001495.jpg",
    "images/ADE_train_00022636.jpg",
    "images/ADE_train_00022635.jpg",
    "images/ADE_frame_00000079.jpg"
]

mask_paths = [
    "images/ADE_000432_aquarium_mask.png",
    "images/instance_001_ADE_train_00022632.png",
    "images/instance_001_ADE_train_00001495.png",
    "images/instance_001_ADE_train_00022636.png",
    "images/instance_007_ADE_train_00022635.png",
    "images/instance_016_ADE_frame_00000079.png"
]

In [67]:
# Initialisiere SimpleLama
simple_lama = SimpleLama()

In [68]:
# Iterationen für Maskenmanipulation
ITERATIONS = 3

# Schleife durch alle Bilder und Masken
for i in range(len(image_paths)):
    # Lade Bild und Maske
    image = Image.open(image_paths[i])
    mask = Image.open(mask_paths[i]).convert('L')

    # Schritt 1: Verenge die Maske (shrink the inpainting region)
    mask = narrow_mask(mask, iterations=ITERATIONS)  # Maske verengen

    # Schritt 2: Progressive Inpainting durch schrittweise Erweiterung der Maske
    for step in range(ITERATIONS + 1):  # Anzahl der progressiven Schritte
        result = simple_lama(image, mask)
        mask = expand_mask(mask, iterations=1)  # Maske schrittweise erweitern
        image = result  # Das Ergebnis als Eingabe für den nächsten Schritt

    # Speichere das finale Ergebnis
    result_path = f"results/expandandnarrowmask_inpainted_progressive_{i+1}_{ITERATIONS}.png"
    result.save(result_path)
    print(f"Ergebnis für Bild {i+1} gespeichert: {result_path}")

Ergebnis für Bild 1 gespeichert: results/expandandnarrowmask_inpainted_progressive_1_3.png
Ergebnis für Bild 2 gespeichert: results/expandandnarrowmask_inpainted_progressive_2_3.png
Ergebnis für Bild 3 gespeichert: results/expandandnarrowmask_inpainted_progressive_3_3.png
Ergebnis für Bild 4 gespeichert: results/expandandnarrowmask_inpainted_progressive_4_3.png
Ergebnis für Bild 5 gespeichert: results/expandandnarrowmask_inpainted_progressive_5_3.png
Ergebnis für Bild 6 gespeichert: results/expandandnarrowmask_inpainted_progressive_6_3.png


In [37]:
'''
# Old code
from simple_lama_inpainting import SimpleLama
from PIL import Image, ImageChops, ImageDraw
import numpy as np
import cv2

# Function to narrow the mask (erosion)
def narrow_mask(mask, iterations=1):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    narrowed_mask = mask_array
    for _ in range(iterations):
        narrowed_mask = cv2.erode(narrowed_mask, kernel, iterations=1)
    return Image.fromarray(narrowed_mask)

# Function to expand the mask (dilation)
def expand_mask(mask, iterations=1):
    mask_array = np.array(mask)
    kernel = np.ones((3, 3), np.uint8)
    expanded_mask = mask_array
    for _ in range(iterations):
        expanded_mask = cv2.dilate(expanded_mask, kernel, iterations=1)
    return Image.fromarray(expanded_mask)

# Initialize SimpleLama
simple_lama = SimpleLama()

# Load image and initial mask
img_path = "images/image.png"
mask_path = "images/mask.png"

image = Image.open(img_path)
mask = Image.open(mask_path).convert('L')

ITERATIONS = 3

# Step 1: Narrow the mask (shrink the inpainting region)
mask = narrow_mask(mask, iterations=ITERATIONS)  # Narrow the mask

# Step 2: Progressive inpainting by expanding the mask
for step in range(ITERATIONS +1):  # Number of progressive steps
    result = simple_lama(image, mask)
    mask = expand_mask(mask, iterations=1)  # Expand mask gradually
    image = result  # Use the result as input for the next step

# Save the final result
result.save(f"results/inpainted_progressive_{ITERATIONS}.png")
'''

In [74]:
# Funktion, um OpenCV-Inpainting durchzuführen
def inpaint_with_opencv(image, mask, inpaint_radius=3, method=cv2.INPAINT_TELEA):
    # Konvertiere das PIL-Image zu einem OpenCV-kompatiblen NumPy-Array
    image_array = np.array(image)
    mask_array = np.array(mask)

    # Sicherstellen, dass Maske binär ist (0 und 255)
    _, mask_array = cv2.threshold(mask_array, 1, 255, cv2.THRESH_BINARY)

    # OpenCV-Inpainting
    inpainted_image = cv2.inpaint(image_array, mask_array, inpaint_radius, method)

    return Image.fromarray(cv2.cvtColor(inpainted_image, cv2.COLOR_BGR2RGB))

In [79]:
# Baseline - Opencv

Defaulting to user installation because normal site-packages is not writeable


In [84]:
from PIL import Image
import os

# Iterationen für Maskenmanipulation
ITERATIONS = 5

# Schleife durch alle Bilder und Masken
for i in range(len(image_paths)):
    # Lade Bild und Maske
    image = Image.open(image_paths[i])
    mask = Image.open(mask_paths[i]).convert('L')
    #mask = expand_mask(mask, iterations=1)

    # OpenCV-Inpainting durchführen
    opencv_result = inpaint_with_opencv(image, mask)

    # Ergebnis speichern
    opencv_result_path = f"results/opencv/opencv_inpainted_{i+1}.png"
    opencv_result.save(opencv_result_path)
    print(f"OpenCV-Inpainting-Ergebnis für Bild {i+1} gespeichert: {opencv_result_path}")


OpenCV-Inpainting-Ergebnis für Bild 1 gespeichert: results/opencv/expandenmask_opencv_inpainted_1.png
OpenCV-Inpainting-Ergebnis für Bild 2 gespeichert: results/opencv/expandenmask_opencv_inpainted_2.png
OpenCV-Inpainting-Ergebnis für Bild 3 gespeichert: results/opencv/expandenmask_opencv_inpainted_3.png
OpenCV-Inpainting-Ergebnis für Bild 4 gespeichert: results/opencv/expandenmask_opencv_inpainted_4.png
OpenCV-Inpainting-Ergebnis für Bild 5 gespeichert: results/opencv/expandenmask_opencv_inpainted_5.png
OpenCV-Inpainting-Ergebnis für Bild 6 gespeichert: results/opencv/expandenmask_opencv_inpainted_6.png


In [ ]:
## Stable Diffusion

In [92]:
import torch
from diffusers import AutoPipelineForInpainting
from diffusers.utils import load_image, make_image_grid
from PIL import Image
import numpy as np

# Pipeline initialisieren
pipeline = AutoPipelineForInpainting.from_pretrained(
    "kandinsky-community/kandinsky-2-2-decoder-inpaint", torch_dtype=torch.float16
)
pipeline.enable_model_cpu_offload()

# Generator mit festem Seed für reproduzierbare Ergebnisse
generator = torch.Generator("cuda").manual_seed(92)

Loading pipeline components...: 100%|██████████| 6/6 [00:00<00:00,  6.23it/s]


In [95]:
# Prompt für das Inpainting
prompt = "empty"

# Ergebnisordner erstellen
result_dir = "results_kandinsky"
os.makedirs(result_dir, exist_ok=True)

# Schleife für Bilder und Masken
for i, (image_path, mask_path) in enumerate(zip(image_paths, mask_paths)):
    # Originalbild und Maske laden
    image_original = load_image(image_path)
    mask_image = load_image(mask_path).convert('L')  # Maske in Graustufen

    #mask_image = expand_mask(mask_image, iterations=1)

 
    # Inpainting mit Stable Diffusion
    result = pipeline(
        prompt=prompt,
        image=image_original,
        mask_image=blurred_mask,
        generator=generator
    ).images[0]

    # Ergebnis speichern
    result_path = f"{result_dir}/kandinsky_inpainted_{i+1}.png"
    result.save(result_path)
    print(f"Stable Diffusion Ergebnis gespeichert: {result_path}")

    # Ergebnisse als Gitter anzeigen (optional)
    grid = make_image_grid([image_original, blurred_mask, result], rows=1, cols=3)
    grid.show()

100%|██████████| 100/100 [00:05<00:00, 19.46it/s]


Stable Diffusion Ergebnis gespeichert: results_kandinsky/with_expand_kandinsky_inpainted_1.png


/bin/xdg-open: 882: www-browser: not found
/bin/xdg-open: 882: links2: not found
/bin/xdg-open: 882: elinks: not found
/bin/xdg-open: 882: links: not found
/bin/xdg-open: 882: lynx: not found
/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpv2y8r0sj.PNG'
100%|██████████| 100/100 [00:05<00:00, 18.87it/s]


Stable Diffusion Ergebnis gespeichert: results_kandinsky/with_expand_kandinsky_inpainted_2.png


/bin/xdg-open: 882: www-browser: not found
/bin/xdg-open: 882: links2: not found
/bin/xdg-open: 882: elinks: not found
/bin/xdg-open: 882: links: not found
/bin/xdg-open: 882: lynx: not found
/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpg5ufmx_d.PNG'
100%|██████████| 100/100 [00:05<00:00, 19.15it/s]


...

/bin/xdg-open: 882: www-browser: not found
/bin/xdg-open: 882: links2: not found
/bin/xdg-open: 882: elinks: not found
/bin/xdg-open: 882: links: not found
/bin/xdg-open: 882: lynx: not found
/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpqp3z9ru0.PNG'
100%|██████████| 100/100 [00:05<00:00, 19.57it/s]


Stable Diffusion Ergebnis gespeichert: results_kandinsky/with_expand_kandinsky_inpainted_5.png


/bin/xdg-open: 882: www-browser: not found
/bin/xdg-open: 882: links2: not found
/bin/xdg-open: 882: elinks: not found
/bin/xdg-open: 882: links: not found
/bin/xdg-open: 882: lynx: not found
/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpddvvfbkj.PNG'
100%|██████████| 100/100 [00:05<00:00, 19.97it/s]


Stable Diffusion Ergebnis gespeichert: results_kandinsky/with_expand_kandinsky_inpainted_6.png


/bin/xdg-open: 882: www-browser: not found
/bin/xdg-open: 882: links2: not found
/bin/xdg-open: 882: elinks: not found
/bin/xdg-open: 882: links: not found
/bin/xdg-open: 882: lynx: not found
/bin/xdg-open: 882: w3m: not found
xdg-open: no method available for opening '/tmp/tmpgn_30i3y.PNG'
